<!--NOTEBOOK_HEADER-->
*This notebook contains material from [CBE30338](https://jckantor.github.io/CBE30338);
content is available [on Github](https://github.com/jckantor/CBE30338.git).*


# Pyomo Examples

In [ ]:
from pyomo.environ import *
import pandas as pd
p

##  Example: Nonlinear Optimization  of Series Reaction in a Continuous Stirred Tank Reactor

In [8]:

V = 40     # liters
kA = 0.5   # 1/min
kB = 0.1   # l/min
CAf = 2.0  # moles/liter

# create a model instance
model = ConcreteModel()

# create x and y variables in the model
model.q = Var()

# add a model objective
model.objective = Objective(expr = model.q*V*kA*CAf/(model.q + V*kB)/(model.q + V*kA), sense=maximize)

# compute a solution using ipopt for nonlinear optimization
results = SolverFactory('ipopt').solve(model)
model.pprint()


# print solutions
qmax = model.q()
CBmax = model.objective()
print('\nFlowrate at maximum CB = ', qmax, 'liters per minute.')
print('\nMaximum CB =', CBmax, 'moles per liter.')
print('\nProductivity = ', qmax*CBmax, 'moles per minute.')

1 Var Declarations
    q : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :  None : 8.944271909985442 :  None : False : False :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40.0*q/(q + 4.0)/(q + 20.0)

2 Declarations: q objective

Flowrate at maximum CB =  8.944271909985442 liters per minute.

Maximum CB = 0.954915028125263 moles per liter.

Productivity =  8.541019662483748 moles per minute.


##  Example: Linear Programming Refinery

In [140]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd
    
# sets 
PRODUCTS = ['Gasoline', 'Kerosine', 'Fuel Oil', 'Residual']
FEEDS = ['Crude 1', 'Crude 2']

# data (2D)
df_products = pd.DataFrame(index=PRODUCTS)
df_products['Price'] = [82, 88, 52, 60]
df_products['Max Production'] = [24000, 2000, 6000, 100000]
df_products['demand'] = [190, 167, 134, 167]

df_crudes = pd.DataFrame(index=FEEDS)
df_crudes['Processing Cost'] = [12, 13]
df_crudes['Feed Costs'] = [48, 30]

# production limits
demand = {'Gasoline': 24000,
                  'Kerosine': 2000,
                  'Fuel Oil': 6000,
                  'Residual': 100000}


# data (3D)
product_yield={}
product_yield[('Gasoline','Crude 1')] = 0.8
product_yield[('Kerosine','Crude 1')] = 0.05
product_yield[('Fuel Oil' ,'Crude 1')] = 0.10
product_yield[('Residual','Crude 1')] = 0.05
product_yield[('Gasoline','Crude 2')] = 0.44
product_yield[('Kerosine','Crude 2')] = 0.10
product_yield[('Fuel Oil','Crude 2')] = 0.36
product_yield[('Residual','Crude 2')] = 0.10


# model formulation
model = pyo.ConcreteModel()

# variables
model.x = pyo.Var(FEEDS,    domain=NonNegativeReals)
model.y = pyo.Var(PRODUCTS, domain=NonNegativeReals)

# objective
income = sum(df_products.loc[p, 'Price'] * model.y[p] for p in PRODUCTS)
raw_materials_cost = sum((df_crudes.loc[f,'Feed Costs'] + df_crudes.loc[f, 'Processing Cost'])  * model.x[f] for f in FEEDS)
profit = income -  raw_materials_cost
model.objective = pyo.Objective(expr = profit, sense=maximize)

    
# constraints
model.constraints = pyo.ConstraintList()
for p in PRODUCTS:
    model.constraints.add(model.y[p] <= demand[p])
    model.constraints.add(model.y[p] == sum(model.x[f] * product_yield[(p,f)] for f in FEEDS))

def pyomo_postprocess(options=None, instance=None, results=None):
    model.x.display()
    model.y.display()
    print('profit:', profit())
    print('income:', income())
    print('raw_materials_cost:', raw_materials_cost())
    print('processing_cost:', processing_cost())

    
#model.pprint()

opt = SolverFactory("glpk")

results = opt.solve(model) # tee=True
# sends results to stdout
#results.write()

    
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('Feasible solution')
    pyomo_postprocess(None, instance, results)
   
    # Do something when the solution in optimal and feasible
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('Infeasible solution')
else:
    print('Status:', results.solver.status)
    # Something else is wrong
    print('termination condition:', results.solver.termination_condition)
    pyomo_postprocess(None, instance, results)
    



Feasible solution
x : Size=2, Index=x_index
    Key     : Lower : Value            : Upper : Fixed : Stale : Domain
    Crude 1 :     0 : 26206.8965517241 :  None : False : False : NonNegativeReals
    Crude 2 :     0 : 6896.55172413793 :  None : False : False : NonNegativeReals
y : Size=4, Index=y_index
    Key      : Lower : Value            : Upper : Fixed : Stale : Domain
    Fuel Oil :     0 : 5103.44827586207 :  None : False : False : NonNegativeReals
    Gasoline :     0 :          24000.0 :  None : False : False : NonNegativeReals
    Kerosine :     0 :           2000.0 :  None : False : False : NonNegativeReals
    Residual :     0 :           2000.0 :  None : False : False : NonNegativeReals
profit: 660413.7931034509
income: 2529379.310344828
raw_materials_cost: 1868965.517241377
processing_cost: 39999.99999999996


##  Example: Linear Programming Refinery with time

In [143]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd
import numpy as np

import numpy as np
PRODUCTS = ['Gasoline', 'Kerosine', 'Fuel Oil', 'Residual']
FEEDS = ['Crude 1', 'Crude 2']
TIME = ['t1', 't2', 't3', 't4']
TANKS = ['o1', 'o2', 'o3', 'o4']

demand = {(t,p): round(1000*np.random.random()) for t in TIME for p in PRODUCTS}

# sets 

# data (2D)
df_products = pd.DataFrame(index=PRODUCTS)
df_products['Price'] = [82, 88, 52, 60]
df_products['Max Production'] = [24000, 2000, 6000, 100000]
df_products['demand'] = [190, 167, 134, 167]


df_crudes = pd.DataFrame(index=FEEDS)
df_crudes['Processing Cost'] = [12, 13]
df_crudes['Feed Costs'] = [48, 30]



# data (3D)
product_yield={}
product_yield[('Gasoline','Crude 1')] = 0.8
product_yield[('Kerosine','Crude 1')] = 0.05
product_yield[('Fuel Oil' ,'Crude 1')] = 0.10
product_yield[('Residual','Crude 1')] = 0.05
product_yield[('Gasoline','Crude 2')] = 0.44
product_yield[('Kerosine','Crude 2')] = 0.10
product_yield[('Fuel Oil','Crude 2')] = 0.36
product_yield[('Residual','Crude 2')] = 0.10


# model formulation
model = pyo.ConcreteModel()

# variables
model.x = pyo.Var(((feed, time) for feed in FEEDS for time in TIME),    domain=NonNegativeReals)
model.y = pyo.Var(((product, time) for product in PRODUCTS for time in TIME), domain=NonNegativeReals)

# objective
income = sum(df_products.loc[p, 'Price'] * model.y[p, t] for p in PRODUCTS for t in TIME)
raw_materials_cost = sum((df_crudes.loc[f,'Feed Costs'] + df_crudes.loc[f, 'Processing Cost'])  * model.x[f, t] for f in FEEDS for t in TIME)
profit = income -  raw_materials_cost
model.objective = pyo.Objective(expr = profit, sense=maximize)

    
# constraints
model.constraints = pyo.ConstraintList()
for p in PRODUCTS:
    for t in TIME:
        model.constraints.add(model.y[p,t] <= demand[t,p])
        model.constraints.add(model.y[p,t] == sum(model.x[f, t] * product_yield[(p,f)] for f in FEEDS))

def pyomo_postprocess(options=None, instance=None, results=None):
    model.x.display()
    model.y.display()
    print('profit:', profit())
    print('income:', income())
    print('raw_materials_cost:', raw_materials_cost())
    

    
#model.pprint()

opt = SolverFactory("glpk")

results = opt.solve(model) # tee=True
# sends results to stdout
#results.write()

    
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('Feasible solution')
    pyomo_postprocess(None, instance, results)
   
    # Do something when the solution in optimal and feasible
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('Infeasible solution')
else:
    print('Status:', results.solver.status)
    # Something else is wrong
    print('termination condition:', results.solver.termination_condition)
    pyomo_postprocess(None, instance, results)
    


Feasible solution
x : Size=8, Index=x_index
    Key               : Lower : Value            : Upper : Fixed : Stale : Domain
    ('Crude 1', 't1') :     0 :            770.0 :  None : False : False : NonNegativeReals
    ('Crude 1', 't2') :     0 :            440.0 :  None : False : False : NonNegativeReals
    ('Crude 1', 't3') :     0 : 195.245901639344 :  None : False : False : NonNegativeReals
    ('Crude 1', 't4') :     0 : 342.786885245901 :  None : False : False : NonNegativeReals
    ('Crude 2', 't1') :     0 : 75.0000000000001 :  None : False : False : NonNegativeReals
    ('Crude 2', 't2') :     0 :              0.0 :  None : False : False : NonNegativeReals
    ('Crude 2', 't3') :     0 : 4.09836065573769 :  None : False : False : NonNegativeReals
    ('Crude 2', 't4') :     0 : 1313.11475409836 :  None : False : False : NonNegativeReals
y : Size=16, Index=y_index
    Key                : Lower : Value            : Upper : Fixed : Stale : Domain
    ('Fuel Oil', 't1') :    

In [144]:
##  Example: Linear Programming Refinery with time and tanks

In [145]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd
import numpy as np

import numpy as np
PRODUCTS = ['Gasoline', 'Kerosine', 'Fuel Oil', 'Residual']
FEEDS = ['Crude 1', 'Crude 2']
TIME = ['t1', 't2', 't3', 't4']
TANKS = ['o1', 'o2', 'o3', 'o4']

demand = {(o,t,p): round(1000*np.random.random()) for o in TANKS for t in TIME for p in PRODUCTS}

# sets 

# data (2D)
df_products = pd.DataFrame(index=PRODUCTS)
df_products['Price'] = [82, 88, 52, 60]
df_products['Max Production'] = [24000, 2000, 6000, 100000]
df_products['demand'] = [190, 167, 134, 167]


df_crudes = pd.DataFrame(index=FEEDS)
df_crudes['Processing Cost'] = [12, 13]
df_crudes['Feed Costs'] = [48, 30]



# data (3D)
product_yield={}
product_yield[('Gasoline','Crude 1')] = 0.8
product_yield[('Kerosine','Crude 1')] = 0.05
product_yield[('Fuel Oil' ,'Crude 1')] = 0.10
product_yield[('Residual','Crude 1')] = 0.05
product_yield[('Gasoline','Crude 2')] = 0.44
product_yield[('Kerosine','Crude 2')] = 0.10
product_yield[('Fuel Oil','Crude 2')] = 0.36
product_yield[('Residual','Crude 2')] = 0.10


# model formulation
model = pyo.ConcreteModel()

# variables
model.x = pyo.Var(((tank, feed, time) for tank in TANKS for feed in FEEDS for time in TIME),    domain=NonNegativeReals)
model.y = pyo.Var(((tank, product, time) for tank in TANKS for product in PRODUCTS for time in TIME), domain=NonNegativeReals)

# objective
income = sum(df_products.loc[p, 'Price'] * model.y[o, p, t] for o in TANKS for p in PRODUCTS for t in TIME)
raw_materials_cost = sum((df_crudes.loc[f,'Feed Costs'] + df_crudes.loc[f, 'Processing Cost'])  * model.x[o, f, t] for o in TANKS for f in FEEDS for t in TIME)
profit = income -  raw_materials_cost
model.objective = pyo.Objective(expr = profit, sense=maximize)

    
# constraints
model.constraints = pyo.ConstraintList()
for o in TANKS:
    for p in PRODUCTS:
        for t in TIME:
            model.constraints.add(model.y[o, p,t] <= demand[o, t,p])
            model.constraints.add(model.y[o, p,t] == sum(model.x[o, f, t] * product_yield[(p,f)] for f in FEEDS))

def pyomo_postprocess(options=None, instance=None, results=None):
    model.x.display()
    model.y.display()
    print('profit:', profit())
    print('income:', income())
    print('raw_materials_cost:', raw_materials_cost())
    

    
#model.pprint()

opt = SolverFactory("glpk")

results = opt.solve(model) # tee=True
# sends results to stdout
#results.write()

    
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('Feasible solution')
    pyomo_postprocess(None, instance, results)
   
    # Do something when the solution in optimal and feasible
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('Infeasible solution')
else:
    print('Status:', results.solver.status)
    # Something else is wrong
    print('termination condition:', results.solver.termination_condition)
    pyomo_postprocess(None, instance, results)
    


Feasible solution
x : Size=32, Index=x_index
    Key                     : Lower : Value            : Upper : Fixed : Stale : Domain
    ('o1', 'Crude 1', 't1') :     0 :            360.0 :  None : False : False : NonNegativeReals
    ('o1', 'Crude 1', 't2') :     0 :              0.0 :  None : False : False : NonNegativeReals
    ('o1', 'Crude 1', 't3') :     0 : 127.586206896552 :  None : False : False : NonNegativeReals
    ('o1', 'Crude 1', 't4') :     0 :              0.0 :  None : False : False : NonNegativeReals
    ('o1', 'Crude 2', 't1') :     0 :              0.0 :  None : False : False : NonNegativeReals
    ('o1', 'Crude 2', 't2') :     0 : 1397.72727272727 :  None : False : False : NonNegativeReals
    ('o1', 'Crude 2', 't3') :     0 : 2036.20689655172 :  None : False : False : NonNegativeReals
    ('o1', 'Crude 2', 't4') :     0 : 1618.18181818182 :  None : False : False : NonNegativeReals
    ('o2', 'Crude 1', 't1') :     0 : 186.896551724138 :  None : False : False : No

##  Example: Linear Programming Refinery (vis)

In [108]:
# problem data
FEEDS = ['Crude #1', 'Crude #2']
PRODUCTS = ['Gasoline', 'Kerosine', 'Fuel Oil', 'Residual']

# feed costs
feed_costs = {'Crude #1': 48,
              'Crude #2': 30}

# processing costs
processing_costs = {'Crude #1': 1.00,
                    'Crude #2': 2.00}

# yield data
product_yield = {('Crude #1', 'Gasoline'): 0.80,
                 ('Crude #1', 'Kerosine'): 0.05,
                 ('Crude #1', 'Fuel Oil'): 0.10,
                 ('Crude #1', 'Residual'): 0.05,
                 ('Crude #2', 'Gasoline'): 0.44,
                 ('Crude #2', 'Kerosine'): 0.10,
                 ('Crude #2', 'Fuel Oil'): 0.36,
                 ('Crude #2', 'Residual'): 0.10}

# product sales prices
sales_price = {'Gasoline': 72,
               'Kerosine': 48,
               'Fuel Oil': 42,
               'Residual': 20}

# production limits
max_production = {'Gasoline': 24000,
                  'Kerosine': 2000,
                  'Fuel Oil': 6000,
                  'Residual': 100000}

# model formulation
model = pyo.ConcreteModel()

# variables
model.x = pyo.Var(FEEDS, domain=NonNegativeReals)
model.y = pyo.Var(PRODUCTS, domain=NonNegativeReals)

# objective
income = sum(sales_price[p] * model.y[p] for p in PRODUCTS)
raw_materials_cost = sum(feed_costs[f] * model.x[f] for f in FEEDS)
processing_cost = sum(processing_costs[f] * model.x[f] for f in FEEDS)
profit = income - raw_materials_cost - processing_cost
model.objective = pyo.Objective(expr = profit, sense=maximize)

# constraints
model.constraints = pyo.ConstraintList()
for p in PRODUCTS:
    model.constraints.add(model.y[p] <= max_production[p])
for p in PRODUCTS:
    model.constraints.add(model.y[p] == sum(model.x[f] * product_yield[(f,p)] for f in FEEDS))

opt = SolverFactory("glpk")

results = opt.solve(model) # tee=True

    
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('Feasible solution')
    pyomo_postprocess(None, instance, results)
    print('profit:', profit())
    print('income:', income())
    print('raw_materials_cost:', raw_materials_cost())
    print('processing_cost:', processing_cost())
    # Do something when the solution in optimal and feasible
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('Infeasible solution')
else:
    print('Status:', results.solver.status)
    # Something else is wrong
    print('termination condition:', results.solver.termination_condition)
    pyomo_postprocess(None, instance, results)

Feasible solution
x : Size=2, Index=x_index
    Key      : Lower : Value            : Upper : Fixed : Stale : Domain
    Crude #1 :     0 : 26206.8965517241 :  None : False : False : NonNegativeReals
    Crude #2 :     0 : 6896.55172413793 :  None : False : False : NonNegativeReals
y : Size=4, Index=y_index
    Key      : Lower : Value            : Upper : Fixed : Stale : Domain
    Fuel Oil :     0 : 5103.44827586207 :  None : False : False : NonNegativeReals
    Gasoline :     0 :          24000.0 :  None : False : False : NonNegativeReals
    Kerosine :     0 :           2000.0 :  None : False : False : NonNegativeReals
    Residual :     0 :           2000.0 :  None : False : False : NonNegativeReals
profit: 573517.241379312
income: 2078344.8275862068
raw_materials_cost: 1464827.5862068948
processing_cost: 39999.99999999996


## Example: Making Change

One of the important modeling features of Pyomo is the ability to index variables and constraints. The

In [117]:
from pyomo.environ import *

def make_change(amount, coins):
    model = ConcreteModel()
    model.x = Var(coins.keys(), domain=NonNegativeIntegers)
    model.total = Objective(expr = sum(model.x[c] for c in coins.keys()), sense=minimize)
    model.amount = Constraint(expr = sum(model.x[c]*coins[c] for c in coins.keys()) == amount)
    SolverFactory('glpk').solve(model)
    return {c: int(model.x[c]()) for c in coins.keys()} 
            
# problem data
coins = {
    'penny': 1,
    'nickel': 5,
    'dime': 10,
    'quarter': 25
}
            
make_change(1034, coins)

{'penny': 4, 'nickel': 1, 'dime': 0, 'quarter': 41}

## Example: Linear Production Model with Constraints with Duals

In [116]:
from pyomo.environ import *
model = ConcreteModel()

# for access to dual solution for constraints
model.dual = Suffix(direction=Suffix.IMPORT)

# declare decision variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(expr = 40*model.x + 30*model.y, sense = maximize)

# declare constraints
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x + model.y <= 80)
model.laborB = Constraint(expr = 2*model.x + model.y <= 100)

# solve
SolverFactory('glpk').solve(model).write()

str = "   {0:7.2f} {1:7.2f} {2:7.2f} {3:7.2f}"

print("Constraint  value  lslack  uslack    dual")
for c in [model.demand, model.laborA, model.laborB]:
    print(c, str.format(c(), c.lslack(), c.uslack(), model.dual[c]))

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2600.0
  Upper bound: 2600.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 6
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.030913591384887695
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------